In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_seq_items = None
pd.options.display.max_rows = None

In [3]:
master = pd.read_csv('../Data/Cleaned/Master/mastermodel_filled_Airbnbfix.csv')

parks = pd.read_csv('../Data/Cleaned/House/house_with_park.csv')

In [4]:
model_df = master.merge(parks[['dist_nearest_park']], left_index=True, right_index=True)

In [72]:
model_df.to_csv('../Data/Cleaned/Master/mastermodel_filledv3.csv', index=False)

In [5]:
model_df.shape

(15234, 167)

In [6]:
model_df.head()

,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,type_Co-Op Apt,type_Co-Ownership Apt,type_Comm Element Condo,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,dist_nearest_school,dist_nearest_ttc,neighbourhood_Agincourt North,neighbourhood_Agincourt South-Malvern West,neighbourhood_Alderwood,neighbourhood_Annex,neighbourhood_Banbury-Don Mills,neighbourhood_Bathurst Manor,neighbourhood_Bay Street Corridor,neighbourhood_Bayview Village,neighbourhood_Bayview Woods-Steeles,neighbourhood_Bedford Park-Nortown,neighbourhood_Beechborough-Greenbrook,neighbourhood_Bendale,neighbourhood_Birchcliffe-Cliffside,neighbourhood_Black Creek,neighbourhood_Blake-Jones,neighbourhood_Briar Hill-Belgravia,neighbourhood_Bridle Path-Sunnybrook-York Mills,neighbourhood_Broadview North,neighbourhood_Brookhaven-Amesbury,neighbourhood_Cabbagetown-South St.James Town,neighbourhood_Caledonia-Fairbank,neighbourhood_Casa Loma,neighbourhood_Centennial Scarborough,neighbourhood_Church-Yonge Corridor,neighbourhood_Clairlea-Birchmount,neighbourhood_Clanton Park,neighbourhood_Cliffcrest,neighbourhood_Corso Italia-Davenport,neighbourhood_Danforth,neighbourhood_Danforth East York,neighbourhood_Don Valley Village,neighbourhood_Dorset Park,neighbourhood_Dovercourt-Wallace Emerson-Junction,neighbourhood_Downsview-Roding-CFB,neighbourhood_Dufferin Grove,neighbourhood_East End-Danforth,neighbourhood_Edenbridge-Humber Valley,neighbourhood_Eglinton East,neighbourhood_Elms-Old Rexdale,neighbourhood_Englemount-Lawrence,neighbourhood_Eringate-Centennial-West Deane,neighbourhood_Etobicoke West Mall,neighbourhood_Flemingdon Park,neighbourhood_Forest Hill North,neighbourhood_Forest Hill South,neighbourhood_Glenfield-Jane Heights,neighbourhood_Greenwood-Coxwell,neighbourhood_Guildwood,neighbourhood_Henry Farm,neighbourhood_High Park North,neighbourhood_High Park-Swansea,neighbourhood_Highland Creek,neighbourhood_Hillcrest Village,neighbourhood_Humber Heights-Westmount,neighbourhood_Humber Summit,neighbourhood_Humbermede,neighbourhood_Humewood-Cedarvale,neighbourhood_Ionview,neighbourhood_Islington-City Centre West,neighbourhood_Junction Area,neighbourhood_Keelesdale-Eglinton West,neighbourhood_Kennedy Park,neighbourhood_Kensington-Chinatown,neighbourhood_Kingsview Village-The Westway,neighbourhood_Kingsway South,neighbourhood_L'Amoreaux,neighbourhood_Lambton Baby Point,neighbourhood_Lansing-Westgate,neighbourhood_Lawrence Park North,neighbourhood_Lawrence Park South,neighbourhood_Leaside-Bennington,neighbourhood_Little Portugal,neighbourhood_Long Branch,neighbourhood_Malvern,neighbourhood_Maple Leaf,neighbourhood_Markland Wood,neighbourhood_Milliken,neighbourhood_Mimico (includes Humber Bay Shores),neighbourhood_Morningside,neighbourhood_Moss Park,neighbourhood_Mount Dennis,neighbourhood_Mount Olive-Silverstone-Jamestown,neighbourhood_Mount Pleasant East,neighbourhood_Mount Pleasant West,neighbourhood_New Toronto,neighbourhood_Newtonbrook East,neighbourhood_Newtonbrook West,neighbourhood_Niagara,neighbourhood_North Riverdale,neighbourhood_North St.James Town,neighbourhood_O'Connor-Parkview,neighbourhood_Oakridge,neighbourhood_Oakwood Village,neighbourhood_Old East York,neighbourhood_Palmerston-Little Italy,neighbourhood_Parkwoods-Donalda,neighbourhood_Pelmo Park-Humberlea,neighbourhood_Playter Estates-Danforth,neighbourhood_Pleasant View,neighbourhood_Princess-Rosethorn,neighbourhood_Regent Park,neighbourhood_Rexdale-Kipling,neighbourhood_Rockcliffe-Smythe,neighbourhood_Roncesvalles,neighbourhood_Rosedale-Moore Park,neighbourhood_Rouge,neighbourhood_Runnymede-Bloor West Village,neighbourhood_Rustic,neighbourhood_Scarborough Village,neighbourhood_South Parkdale,neighbourhood_South Riverdale,neighbourhood_St.Andrew-Windfields,neighbourhood_Steeles,neighbourhood_Stonegate-Queensway,neighbourhood_Tam O'Shanter-Sullivan,neighbourhood_Taylor-Massey,neighbourhood_The Beach

## *Creating an ensemble model* 

In [25]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score
from lightgbm import LGBMRegressor

*create train test split*

In [8]:
predictors = model_df.drop(columns=['final_price'], axis=1).columns

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(model_df[predictors], model_df['final_price'], test_size=0.30, random_state=42)

*model function*

In [15]:
def modelfit(pip, x_train, y_train, x_test, y_test, predictors, cv_folds=5):
    
    
    #Fit the algorithm on the data
    pip.fit(x_train, y_train)
    
    y_val = y_test.values
    
    #Perform cross-validation:
    #if performCV:
    #cv_score = cross_val_score(pip, x_train, y_train, cv=cv_folds, scoring='neg_mean_squared_log_error')
    
    #Predict test set:
    pred = pip.predict(x_test)
    
    #Print model report:
    print('Model Report')
    print(f'Accuracy : {round(mean_squared_log_error(y_val, pred),3)}')
    
    #if performCV:
    #print(f'CV Score : Mean - {np.mean(cv_score)} | Std - {np.std(cv_score)} | Min - {np.min(cv_score)} | Max - {np.max(cv_score)}')
                
    #Print Feature Importance:
    #if printFeatureImportance:
    #feat_imp = pd.Series(pip.steps[1][1].feature_importances_, predictors).sort_values(ascending=False)
    #feat_imp.head(20).plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')
    data = {'Actual': y_val,
           'Predictions': pred}
    return pd.DataFrame(data)

In [11]:
def create_pipeline(scale, model):
    steps = [('std',scale), ('model', model)]
    return Pipeline(steps=steps)

#### Random Forest Regressor

In [16]:
rf = RandomForestRegressor()
scale = StandardScaler()

pipeline_rf = create_pipeline(scale, rf)

op_rf = modelfit(pipeline_rf, X_train, Y_train, X_test, Y_test,predictors)

Model Report
Accuracy : 0.021


#### Light GBM

In [24]:
lgb = LGBMRegressor()

pipeline_lgb = create_pipeline(scale, lgb)

op_lgb = modelfit(pipeline_lgb, X_train, Y_train, X_test, Y_test, predictors)

Model Report
Accuracy : 0.025


#### ExtraTrees Regressor

In [26]:
ext = ExtraTreesRegressor()

pipeline_ext = create_pipeline(scale, ext)

op_ext = modelfit(pipeline_ext, X_train, Y_train, X_test, Y_test, predictors)

Model Report
Accuracy : 0.017


#### *Parameter Tunning in ExtraTreesRegresor*

In [27]:
ExtraTreesRegressor().get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [28]:
from sklearn.model_selection import GridSearchCV

In [95]:
g_ext = ExtraTreesRegressor(n_estimators=140, max_depth=15, min_samples_split=4, criterion = 'mse', random_state = 10)
                            

pipelineGSV = create_pipeline(scale, g_ext)

param_test = {'model__bootstrap':[False, True]}

g_search = GridSearchCV(pipelineGSV, param_test,scoring='neg_mean_squared_log_error',n_jobs=None, cv=5,refit=False)

In [96]:
g_search.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('std', StandardScaler()),
                                       ('model',
                                        ExtraTreesRegressor(max_depth=15,
                                                            min_samples_split=4,
                                                            n_estimators=140,
                                                            random_state=10))]),
             param_grid={'model__bootstrap': [False, True]}, refit=False,
             scoring='neg_mean_squared_log_error')

In [97]:
g_search.cv_results_, g_search.best_params_, g_search.best_score_

({'mean_fit_time': array([9.64880872, 7.02350698]),
  'std_fit_time': array([0.76887289, 0.53993951]),
  'mean_score_time': array([0.07289228, 0.07857413]),
  'std_score_time': array([0.00786048, 0.03250506]),
  'param_model__bootstrap': masked_array(data=[False, True],
               mask=[False, False],
         fill_value='?',
              dtype=object),
  'params': [{'model__bootstrap': False}, {'model__bootstrap': True}],
  'split0_test_score': array([-0.0215091 , -0.02183852]),
  'split1_test_score': array([-0.0212592 , -0.02200616]),
  'split2_test_score': array([-0.01982995, -0.02059338]),
  'split3_test_score': array([-0.02065457, -0.02120457]),
  'split4_test_score': array([-0.02294893, -0.02371861]),
  'mean_test_score': array([-0.02124035, -0.02187225]),
  'std_test_score': array([0.00103191, 0.00104971]),
  'rank_test_score': array([1, 2], dtype=int32)},
 {'model__bootstrap': False},
 -0.021240351507526477)

## *Stacking Models*

In [107]:
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [108]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('rf', RandomForestRegressor(random_state=10)))
    level0.append(('ext', ExtraTreesRegressor(n_estimators=140, max_depth=15, min_samples_split=4, random_state = 10)))
    level0.append(('lgb', LGBMRegressor(random_state=10)))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model

In [110]:
# get a list of models to evaluate
def get_models():
    models = dict()
    #models['rf'] = RandomForestRegressor(random_state=10)
    #models['ext'] = ExtraTreesRegressor(n_estimators=140, max_depth=15, min_samples_split=4, random_state = 10)
    #models['lgb'] = LGBMRegressor(random_state=10)
    models['stacking'] = get_stacking()
    return models

In [99]:
# evaluate a given model using cross-validation
def evaluate_model(pipeline, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_log_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [111]:
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    pip = create_pipeline(scale, model)
    scores = evaluate_model(pip, X_train, Y_train)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))

# plot model performance for comparison
#pyplot.boxplot(results, labels=names, showmeans=True)
#pyplot.show()

>stacking -0.021 (0.002)
